In [3]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.metrics import precision_recall_fscore_support
%matplotlib inline

In [16]:
def read_dataset(filePath,delimiter=','):
    return pd.read_csv(filePath)

def reshape_data(features,labels):
    n_training_samples = features.shape[0]
    n_dim = features.shape[1]
    f = np.asarray(np.reshape(features,[n_training_samples,n_dim]))
    l = np.asarray(labels)
    return f, l

In [17]:
#Dataset can be acquired from UCI ML repository (https://archive.ics.uci.edu/ml/datasets/Letter+Recognition)
data = read_dataset('letter-recognition.csv')
features = data.ix[:,0:-1]
labels = pd.get_dummies(data.ix[:,-1])
f, l = reshape_data(features,labels)
n_dim = f.shape[1]

rnd_indices = np.random.rand(len(f)) < 0.70
train_x = f[rnd_indices]
train_y = l[rnd_indices]
test_x = f[~rnd_indices]
test_y = l[~rnd_indices]

In [19]:
train_features = tf.placeholder(tf.float32,[None,n_dim]) #train
test_features = tf.placeholder(tf.float32,[None]) #test
init = tf.initialize_all_variables()

In [20]:
euc_distance = tf.sqrt(tf.reduce_sum(tf.squared_difference(train_features,test_features),reduction_indices=1))
nearest_neighbour = tf.argmin(euc_distance, 0)

In [21]:
accuracy = 0
y_true = []
y_pred = []

with tf.Session() as sess:
    sess.run(init)
    for i in range(len(test_x)):
        dists = sess.run(euc_distance,feed_dict={train_features:train_x,test_features:test_x[i,:]})
        nn_idx = sess.run(tf.argmin(dists, 0))
        y_true.append(np.argmax(test_y[i]))
        y_pred.append(np.argmax(train_y[nn_idx]))

In [26]:
p,r,fscore,s = precision_recall_fscore_support(y_true, y_pred, average='micro')

In [27]:
print fscore

0.954476222711
